<a href="https://colab.research.google.com/github/VictoriaRe/HSE-DataScience/blob/master/assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import warnings
warnings.filterwarnings('ignore')

In [0]:
import nltk

In [23]:
import pandas as pd
df = pd.read_csv('imdb_master.csv', encoding = 'latin-1')
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [24]:
df_test = df[df['type']=='test'].drop(['Unnamed: 0','type','file'],axis=1)
df_train = df[df['type']=='train'].drop(['Unnamed: 0','type','file'],axis=1)
len(df_test), len(df_train)

(25000, 75000)

In [25]:
df_train.head()

,review,label
25000,Story of a man who has unnatural feelings for ...,neg
25001,Airport '77 starts as a brand new luxury 747 p...,neg
25002,This film lacked something I couldn't put my f...,neg
25003,"Sorry everyone,,, I know this is supposed to b...",neg
25004,When I was little my parents took me along to ...,neg


In [0]:
df_train = df_train[df_train['label'] != 'unsup']
df_test = df_test[df_test['label'] != 'unsup']

In [27]:
len(df_test), len(df_train)

(25000, 25000)

In [0]:
df_train['label'] = df_train['label'].apply(lambda x : 0 if x=='neg' else 1)
df_test['label'] = df_test['label'].apply(lambda x : 0 if x=='neg' else 1)

In [33]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Preprocess text

In [35]:
nltk.download('wordnet')
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
#     lower letters
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
#     delete stopwords
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df_train['processed'] = df_train.review.apply(lambda x: preprocess(x))
df_test['processed'] = df_test.review.apply(lambda x: preprocess(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Deep Model

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_train['processed'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['processed'])

maxlen = 150
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_train['label']

embed_size = 256
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 200
epochs = 3


In [39]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)




Train on 20000 samples, validate on 5000 samples
Epoch 1/3





20000/20000 [==============================] - 177s 9ms/step - loss: 0.4586 - acc: 0.7713 - val_loss: 0.5704 - val_acc: 0.7540
Epoch 2/3
20000/20000 [==============================] - 174s 9ms/step - loss: 0.2083 - acc: 0.9220 - val_loss: 0.5850 - val_acc: 0.7588
Epoch 3/3
20000/20000 [==============================] - 175s 9ms/step - loss: 0.1241 - acc: 0.9575 - val_loss: 0.9844 - val_acc: 0.6734


### Check accuracy on test data

In [0]:
list_tokenized_test = tokenizer.texts_to_sequences(df_test['processed'])
maxlen = 150
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [41]:
df_test.head()

,review,label,processed
0,Once again Mr. Costner has dragged out a movie...,0,mr costner ha drag movie far longer necessary ...
1,This is an example of why the majority of acti...,0,example majority action film generic bore real...
2,"First of all I hate those moronic rappers, who...",0,first hate moronic rapper couldnt act gun pres...
3,Not even the Beatles could write songs everyon...,0,even beatles could write song everyone like al...
4,Brass pictures (movies is not a fitting word f...,0,brass picture movie fit word really somewhat b...


In [0]:
def take_pred(x):
    if x>=0.5:
        return 1
    else:
        return 0
preds = [take_pred(x[0]) for x in model.predict(X_test)]

In [43]:
(preds == df_test['label'].values).mean()

0.82788



```
# This is formatted as code
```

### Получили 83% точности! Но мы видим, что сеть переобучается(судя по скору val_accuracy). Давайте попробуем сделаем сеть попроще.

In [0]:
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_train['processed'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['processed'])

maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_train['label']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.15))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 200
epochs = 3

In [45]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 63s 3ms/step - loss: 0.5391 - acc: 0.7134 - val_loss: 0.4860 - val_acc: 0.7976
Epoch 2/3
20000/20000 [==============================] - 61s 3ms/step - loss: 0.2725 - acc: 0.8921 - val_loss: 0.5124 - val_acc: 0.7798
Epoch 3/3
20000/20000 [==============================] - 61s 3ms/step - loss: 0.2013 - acc: 0.9255 - val_loss: 0.5932 - val_acc: 0.7590


### Score

In [46]:
list_tokenized_test = tokenizer.texts_to_sequences(df_test['processed'])
maxlen = 100
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
preds = [take_pred(x[0]) for x in model.predict(X_test)]
(preds == df_test['label'].values).mean()

0.845

## Получилось лучше: 85%. Попробуем использовать unsupervised dataset.

##### Что мы сделаем? Классифицируем unsupervised с помощью старой модели и обучем по-новому

In [0]:
df_unsup = df[df['type']=='train'].drop(['Unnamed: 0','type','file'],axis=1)
df_unsup = df_unsup[df_unsup['label'] == 'unsup']

df_unsup['label'] = df_unsup['label'].apply(lambda x : 0 if x=='neg' else 1)
df_unsup['processed'] = df_unsup.review.apply(lambda x: preprocess(x))

In [0]:
list_tokenized_unsup = tokenizer.texts_to_sequences(df_unsup['processed'])
maxlen = 100
X_unsup = pad_sequences(list_tokenized_unsup, maxlen=maxlen)
df_unsup['label'] = [take_pred(x[0]) for x in model.predict(X_unsup)]

In [0]:
df_new = pd.concat([df_unsup, df_train]).reset_index(drop=True)

max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_new['processed'])
list_tokenized_train = tokenizer.texts_to_sequences(df_new['processed'])

maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_new['label']

embed_size = 256
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 200
epochs = 3

In [50]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 1237s 21ms/step - loss: 0.3148 - acc: 0.8573 - val_loss: 0.3494 - val_acc: 0.8599
Epoch 2/3
60000/60000 [==============================] - 1237s 21ms/step - loss: 0.1621 - acc: 0.9357 - val_loss: 0.4488 - val_acc: 0.8357
Epoch 3/3
60000/60000 [==============================] - 1239s 21ms/step - loss: 0.1303 - acc: 0.9493 - val_loss: 0.9078 - val_acc: 0.6949


### Score

In [51]:
list_tokenized_test = tokenizer.texts_to_sequences(df_test['processed'])
maxlen = 100
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
preds = [take_pred(x[0]) for x in model.predict(X_test)]
(preds == df_test['label'].values).mean()

0.7878

In [52]:
c = 0
for i in range(len(df_test)):
    if df_test.loc[i, 'label'] != preds[i]:
        print(df_test.loc[i, 'processed'])
        print(' ---------------------------------- ')
        c += 1
    if (c == 10):
        break

think would appropriate title film since pad hell back stock footage various bug animal recently find prey original vhs big box form wa excite love find old slasher film vhs cover artwork fantastic usually though turn film le fantastic prey one filmsbr br fair start ok killer stalk clichã teenager wood heartbeat sound use great effect make tense watch film basically big fat clichã campfire story section roll film take new direction spend almost half run time backstory killer actually think wa quite original idea however backstory end abruptly show u stockfootage burn woodland lack budget really start show now). this, return dumb teenager pick woods. killer show end, shame actually make effective look killer. sort like cropsy burning, better. gore, much, although there's ok face squish moment end. <br /><br />overall, recommend film anyone slasher completists - really big mess.
 ---------------------------------- 
one strange selfimportant selfindulgent movie try hard profound isnt inst

In [53]:
model.fit(X_t,y, batch_size=batch_size, epochs=3, validation_split=0.2)

Train on 60000 samples, validate on 15000 samples
Epoch 1/3
60000/60000 [==============================] - 1240s 21ms/step - loss: 0.1149 - acc: 0.9553 - val_loss: 0.7251 - val_acc: 0.7935
Epoch 2/3
60000/60000 [==============================] - 1246s 21ms/step - loss: 0.0755 - acc: 0.9719 - val_loss: 0.5777 - val_acc: 0.8451
Epoch 3/3
60000/60000 [==============================] - 1249s 21ms/step - loss: 0.0556 - acc: 0.9796 - val_loss: 0.7081 - val_acc: 0.8320


### Score

In [54]:
preds = [take_pred(x[0]) for x in model.predict(X_test)]
(preds == df_test['label'].values).mean()

0.83684